<a href="https://colab.research.google.com/github/HyeonminNam/TIL/blob/master/NLP_C/200721_Text_Summarize_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Summarize

In [1]:
import pandas as pd
import numpy as np
import copy

In [2]:
# 초기입력값
text = '딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나.'
damp = 0.85
threshold = 0.001

### 문장 분리

In [3]:
sent_lst = [x.strip().split() for x in text.split('.')[:-1]]

In [4]:
sent_lst

[['딸기', '바나나', '사과', '파인애플', '수박'],
 ['바나나', '사과', '딸기', '포도'],
 ['복숭아', '수박'],
 ['파인애플', '사과', '딸기', '바나나']]

### 각 문장 단어 분리

In [5]:
token_lst = [set(x) for x in sent_lst]

In [13]:
token_lst

[{'딸기', '바나나', '사과', '수박', '파인애플'},
 {'딸기', '바나나', '사과', '포도'},
 {'복숭아', '수박'},
 {'딸기', '바나나', '사과', '파인애플'}]

### 문장간 유사도 계산

In [14]:
sim_mat = np.zeros((len(token_lst), len(token_lst)))
for idx_1, tokens in enumerate(token_lst):
    for idx_2, other in enumerate(token_lst):
        if idx_1 != idx_2:
            sim = len(tokens.intersection(other))/len(tokens.union(other))
            sim_mat[idx_1, idx_2] = sim

In [15]:
sim_mat

array([[0.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 0.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 0.        , 0.        ],
       [0.8       , 0.6       , 0.        , 0.        ]])

### 엣지 가중치, 최초 스코어 계산

In [16]:
edge = np.zeros((len(token_lst), len(token_lst)))
score_lst = []
for idx_1, tokens in enumerate(token_lst):
    total = np.sum(sim_mat[idx_1])
    score_lst.append(total)
    for idx_2, other in enumerate(token_lst):
        if idx_1 != idx_2:
            edge[idx_1, idx_2] = sim_mat[idx_1, idx_2]/total        

In [17]:
edge

array([[0.        , 0.34090909, 0.11363636, 0.54545455],
       [0.45454545, 0.        , 0.        , 0.54545455],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142857, 0.42857143, 0.        , 0.        ]])

In [18]:
score_lst

[1.4666666666666668, 1.1, 0.16666666666666666, 1.4]

In [19]:
sim_mat

array([[0.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 0.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 0.        , 0.        ],
       [0.8       , 0.6       , 0.        , 0.        ]])

### 새로운 스코어, 문장 유사도 계산

In [20]:
damp_np = np.array([damp for x in range(len(token_lst))])
while True:
    prev_score = copy.copy(score_lst)
    for idx_1, score in enumerate(prev_score):
        for idx_2, other in enumerate(prev_score):
            if edge[idx_1, idx_2] != 0:
                sim = score*edge[idx_1][idx_2]
                sim_mat[idx_1, idx_2] = sim
    new_score = damp_np.dot(sim_mat) + (1-damp)
    score_lst = copy.copy(new_score)
    print(sim_mat)
    if np.sum(np.fabs(prev_score-new_score)) <= threshold:
        print('최종 스코어: ')
        print(score_lst)
        break

[[0.         0.5        0.16666667 0.8       ]
 [0.5        0.         0.         0.6       ]
 [0.16666667 0.         0.         0.        ]
 [0.8        0.6        0.         0.        ]]
[[0.         0.47613636 0.15871212 0.76181818]
 [0.49318182 0.         0.         0.59181818]
 [0.29166667 0.         0.         0.        ]
 [0.76571429 0.57428571 0.         0.        ]]
[[0.         0.50044717 0.16681572 0.80071547]
 [0.47402671 0.         0.         0.56883205]
 [0.2849053  0.         0.         0.        ]
 [0.74319481 0.5573961  0.         0.        ]]
[[0.         0.48641175 0.16213725 0.7782588 ]
 [0.47689399 0.         0.         0.57227279]
 [0.29179336 0.         0.         0.        ]
 [0.75092308 0.56319231 0.         0.        ]]
[[0.         0.49147802 0.16382601 0.78636484]
 [0.47371066 0.         0.         0.56845279]
 [0.28781666 0.         0.         0.        ]
 [0.74168677 0.55626508 0.         0.        ]]
[[0.         0.48672681 0.16224227 0.7787629 ]
 [0.4729

In [21]:
score_dic = {'스코어': score_lst}
idx = ['문장{}'.format(x+1) for x in range(len(score_lst))]

In [22]:
score_dic

{'스코어': array([1.41215533, 1.02593219, 0.28643249, 1.28064793])}

In [23]:
df = pd.DataFrame(score_dic, index= idx)

In [24]:
df

,스코어
문장1,1.412155
문장2,1.025932
문장3,0.286432
문장4,1.280648


### 클래스 만들기

In [36]:
import pandas as pd
import numpy as np
import copy

# 입력값(텍스트, 윈도우 사이즈)
class TextSummarize:
    
    def __init__(self, text):
        self.text = text
        self.sent_lst = [x.strip().split() for x in self.text.split('.')[:-1]]
        self.token_lst = [set(x) for x in self.sent_lst]
        self.score_lst = []

    def edge(self):
        # 문장 유사도 계산
        sim_mat = np.zeros((len(token_lst), len(token_lst)))
        for idx_1, tokens in enumerate(token_lst):
            for idx_2, other in enumerate(token_lst):
                if idx_1 != idx_2:
                    sim = len(tokens.intersection(other))/len(tokens.union(other))
                    sim_mat[idx_1, idx_2] = sim
                    
        # 엣지, 최초 스코어 계산
        edge = np.zeros((len(token_lst), len(token_lst)))
        score_lst = []
        for idx_1, tokens in enumerate(token_lst):
            total = np.sum(sim_mat[idx_1])
            score_lst.append(total)
            for idx_2, other in enumerate(token_lst):
                if idx_1 != idx_2:
                    edge[idx_1, idx_2] = sim_mat[idx_1, idx_2]/total
        self.score_lst = score_lst
        return edge
                    

    def score(self, threshold, damp):
        # 행렬 계산 위해서 damp 값으로 (4, 0) 행렬 만들기
        damp_np = np.array([damp for x in range(len(token_lst))])
        
        # threshold까지 반복해서 스코어 갱신
        while True:
            prev_score = copy.copy(self.score_lst)
            for idx_1, score in enumerate(prev_score):
                for idx_2, other in enumerate(prev_score):
                    if edge[idx_1, idx_2] != 0:
                        sim = score*edge[idx_1][idx_2]
                        sim_mat[idx_1, idx_2] = sim
            new_score = damp_np.dot(sim_mat) + (1-damp)
            self.score_lst = copy.copy(new_score)
            
            # threshold조건 도달하면 데이터프레임 리턴
            if np.sum(np.fabs(prev_score-new_score)) <= threshold:
                print('최종 스코어: ')
                print(self.score_lst)
                df = pd.DataFrame(score_dic, index= idx)
                return df

## TextSummarize 클래스 테스트

In [37]:
text = '딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나.'
damp = 0.85
threshold = 0.001

In [38]:
test = TextSummarize(text)

In [39]:
test.edge()

array([[0.        , 0.34090909, 0.11363636, 0.54545455],
       [0.45454545, 0.        , 0.        , 0.54545455],
       [1.        , 0.        , 0.        , 0.        ],
       [0.57142857, 0.42857143, 0.        , 0.        ]])

In [40]:
test.score(0.001, 0.85)

최종 스코어: 
[1.41215533 1.02593219 0.28643249 1.28064793]


,스코어
문장1,1.412155
문장2,1.025932
문장3,0.286432
문장4,1.280648
